<a href="https://colab.research.google.com/github/Dichik/gathers-nlp-final-project/blob/main/src/website_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

Parse manipulation news from https://disinfo.detector.media/theme/manipulation

Note: there are 17 pages.

In [2]:
import time
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup, SoupStrainer

In [3]:
url = 'https://disinfo.detector.media/theme/manipulation'

sub_headers = []
h2s = []

fake_news = set()
true_news = set()

In [15]:
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

print(response.content.decode())
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser', parse_only=SoupStrainer('article'))
    for article in soup: 
      if article.has_attr('a'):
        print(article)
        break
else:
    raise Exception('Failed to fetch the page')

<!DOCTYPE html>
<html lang="uk">
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">

    <!-- CSRF Token -->
    <meta name="csrf-token" content="GAC8lvQ5U4aqSR1i0dswnyY2IQiK9BriAk1fTBwT">
        <title> 831 - Маніпуляція | #DisinfoChronicle.</title>

    <meta property="og:url" content="https://disinfo.detector.media/theme/manipulation" />
    <meta property="og:locale" content="uk" />

            <meta property="og:type" content="website" />
        <meta name="twitter:card" content="summary_large_image">
        <meta property="og:title" content="#DisinfoChronicle. Кремлівська дезінформація щодо військового наступу на Україну - Детектор медіа." />
        <meta name="twitter:title" content="#DisinfoChronicle. Кремлівська дезінформація щодо військового наступу на Україну - Детектор медіа.">

        <meta name="description" content="&amp;laquo;Детектор медіа&amp;raquo; 

In [22]:
news = soup.find_all(class_='newsFull')
len(news)

25

In [60]:
def get_manipulative_title(news):
  manipulative_title = news.find(class_='newsFull__title').text
  manipulative_title = manipulative_title[15:]
  return manipulative_title.strip()

one_news = news[2]
get_manipulative_title(one_news)

'У кількості танків, які Німеччина надасть Україні, зашифрували нацистські символи'

In [66]:
def get_manipulative_and_real_content(news):
  manipulative_content = news.find(class_='newsFull__content').text
  arrs = manipulative_content.split('Це — маніпуляція')
  manipulative = arrs[0]
  real = arrs[1] if len(arrs) > 1 else ''
  return manipulative, real

one_news = news[0]
get_manipulative_and_real_content(one_news)

('\nУ соціальних мережах поширюють фото та відео з Сейму Литви. На них зображені хористи у вишиванках, що виконують гімн України, текст якого надрукований на аркушах з українським прапором та гербом на звороті.\xa0У дописах йдеться, що це свідчення того, що Україна вже окупувала не лише Литву, а й інші країни Балтії. ',
 '.\nУ Сеймі справді виконували гімн України на знак солідарності з Україною 24 лютого 2023 року. Фактчекери проєкту Delfi за допомогою програмного забезпечення підтвердили час і місце фільмування. Заходи, присвячені висловленню солідарності з Україною, під час яких використовували\xa0 українську символіку, відбувалися в багатьох країнах світу. Тож це ніяк не можна розцінювати як доказ окупації Україною країн Балтії. Фактчекери зазначають, що це не перший випадок спотворення інформації про використання української символіки чи елементів української\xa0 культури.\xa0\nАгітпроп навмисно поширює меседжі про «українізацію» держав, які допомагають Україні, щоб налаштувати їх

In [67]:
def parse_page(page):
  url = f'https://disinfo.detector.media/theme/manipulation?page={page}'

  response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})


  if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser', parse_only=SoupStrainer('article'))
      news = soup.find_all(class_='newsFull')

      for n in news:
        # parse main content with real and manipulative news
        manipulative, real = get_manipulative_and_real_content(n)
        real_news.append(real)
        manipulative_news.append(manipulative)

        # parse manipulate titles
        manipulative_news.append(get_manipulative_title(n))
  else:
      raise Exception('Failed to fetch the page')

In [68]:
real_news = []
manipulative_news = []

for page in range(18):
  parse_page(page)

In [69]:
len(real_news)

445

In [70]:
len(manipulative_news)

890

In [77]:
def light_preprocess(text):
  text = text.replace('\n', '')
  text = text.replace('.', ' ')
  text = text.strip()
  return text

In [78]:
reals = pd.DataFrame(real_news, columns=['Message'])
reals['Message'] = reals['Message'].apply(light_preprocess)
reals.head()

,Message
0,У Сеймі справді виконували гімн України на зна...
1,"Італійський рис справді потрапив до України, о..."
2,Твердження про зашифровані символи у кількості...
3,На неї звернули увагу аналітики проєкту VoxChe...
4,На неї звернули увагу аналітики проєкту The In...


In [79]:
manipulatives = pd.DataFrame(manipulative_news, columns=['Message'])
manipulatives['Message'] = manipulatives['Message'].apply(light_preprocess)
manipulatives.head()

,Message
0,У соціальних мережах поширюють фото та відео з...
1,Україна окупувала Литву та інші країни Балтії
2,"У соціальних мережах поширюють інформацію, ніб..."
3,В Україну завезли «забракований» в Італії рис
4,"У мережі поширюють повідомлення, що нібито кіл..."


In [83]:
reals.to_csv('reals.csv')

In [84]:
manipulatives.to_csv('manipulatives.csv')